In [1]:
import os
from sentence_transformers import SentenceTransformer, losses, BiSentenceTransformer
from sentence_transformers.readers import STSDataReader, FEVERReader
from sentence_transformers.datasets import SentencesDataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import psycopg2
import time
import numpy as np

## Training with Siamese Model

In [2]:
base_model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
train_batch_size = 16
num_epochs = 1
warmup_steps=100
model_save_path='./fever-model'

In [3]:
reader = FEVERReader()
train_examples = reader.get_examples('train',table='test.train_article_rerank')
train_data = SentencesDataset(train_examples, base_model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=base_model)

dev_examples = reader.get_examples('dev',table='test.test_article_rerank')
dev_data = SentencesDataset(examples=dev_examples, model=base_model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          fp16=True,
          fp16_opt_level='O1'
)

trying to connect to postgres...
connected to postgres
downloading data
trying to connect to postgres...
connected to postgres
downloading data


NameError: name 'model' is not defined

## Training with Modified Siamese (dense feedforward layer for query)

In [4]:
model = BiSentenceTransformer(base_model)
train_batch_size = 16
num_epochs = 1
warmup_steps=100
model_save_path='./modified-fever'

In [4]:
reader = FEVERReader()
train_examples = reader.get_examples('train',table='test.train_article_rerank')
train_data = SentencesDataset(train_examples, base_model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.BiCosineSimilarityLoss(model=model)

dev_examples = reader.get_examples('dev',table='test.test_article_rerank')
dev_data = SentencesDataset(examples=dev_examples, model=base_model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)

trying to connect to postgres...
connected to postgres
downloading data
trying to connect to postgres...
connected to postgres
downloading data


In [ ]:
model.fit((train_dataloader, train_loss),
          None,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path_base=model_save_path,
          fp16=True,
          fp16_opt_level='O1'
)

Iteration:   0%|          | 0/6823 [00:00<?, ?it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/opt/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:73: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)

Iteration:   2%|▏         | 120/6823 [00:24<21:57,  5.09it/s]


Iteration:   4%|▎         | 246/6823 [00:49<20:15,  5.41it/s]


Iteration:   5%|▌         | 372/6823 [01:14<20:21,  5.28it/s]


...

Iteration:  20%|██        | 1374/6823 [04:32<17:48,  5.10it/s]


Iteration:  22%|██▏       | 1498/6823 [04:57<18:06,  4.90it/s]


Iteration:  24%|██▍       | 1622/6823 [05:22<18:02,  4.81it/s]


Iteration:  26%|██▌       | 1746/6823 [05:46<16:34,  5.10it/s]


Iteration:  27%|██▋       | 1843/6823 [06:05<16:21,  5.07it/s]


In [13]:
import torch
save_path = f'{model_save_path}/Query'
os.makedirs(save_path, exist_ok=True)
torch.save(model.model_b.linear.state_dict(), os.path.join(save_path, 'pytorch_model.bin'))

In [14]:
load_path = os.path.join(model_save_path, 'Query', 'pytorch_model.bin')
model.model_b.linear.load_state_dict(torch.load(load_path))

<All keys matched successfully>

In [4]:
HOST = '54.196.150.193'
USER = 'postgres'
PASS = os.environ.get('PGPASS')
PGSSLROOTCERT = os.environ.get('PGSSLROOTCERT')
if PASS == None or PGSSLROOTCERT == None:
    print("Please set PG_PASS and PGSSLROOTCERT env variable")
    raise SystemExit()
DBNAME = 'fever'
POSTGRES_DSN = f'''dbname='fever' user='{USER}' host='{HOST}' password='{PASS}' '''
model = SentenceTransformer('./fever-model/')

In [24]:
query = '''
select a.id, l.text 
from wiki.articles a
join wiki.lines l on l.article_id = a.id and line_number = 0
'''

In [25]:
conn = psycopg2.connect(POSTGRES_DSN)
cur = conn.cursor()
cur.execute(query)
res = cur.fetchall()

In [ ]:
sent_buffer = []
ids_buffer = []
BATCH_SIZE = 100000
if not os.path.exists('./fever-embs/'):
    os.makedirs('./fever-embs/')
start = time.time()
for i, batch in enumerate(res):
    sent_buffer.append(batch[1])
    ids_buffer.append(batch[0])
    if (i+1) % BATCH_SIZE == 0:
        embs = model.encode(sent_buffer, batch_size=32)
        ids_buffer = np.array(ids_buffer)
        ids_buffer = np.expand_dims(ids_buffer, 1)
        to_save = np.concatenate((ids_buffer, embs), 1)
        np.save(f'./fever-embs/emb-{i}',to_save)
        sent_buffer = []
        ids_buffer = []
        print(f'Running {i/(time.time() - start)} per second')

Running 339.2325537255288 per second


In [29]:
#embs = model.encode(sent_buffer, batch_size=32)
#ids_buffer = np.array(ids_buffer)
#ids_buffer = np.expand_dims(ids_buffer, 1)
#to_save = np.concatenate((ids_buffer, embs), 1)
np.save(f'./fever-embs/emb-last',to_save)